In [1]:
import pandas as pd
import numpy as np
import random
sys.path.insert(0, '/home/lizian/.local/lib/python3.7/site-packages') 
# need to change to your own installation path so that you can import wrds

import wrds

In [2]:
# get connected
db = wrds.Connection(wrds_username='xiaoy5')

Enter your WRDS username [lizian]:xiaoy5
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
There was an error with your password.


OperationalError: (psycopg2.OperationalError) could not connect to server: Connection timed out
	Is the server running on host "wrds-pgdata.wharton.upenn.edu" (165.123.60.118) and accepting
	TCP/IP connections on port 9737?

(Background on this error at: http://sqlalche.me/e/e3q8)

In [3]:
CIK_list = tuple(["0000320193", "0000789019"])
# set date limit
DATE_list = tuple(["2000-01-01", "2018-12-30"])
FY = 2000

In [4]:
# Execute sql commands, extract the items with date between 2000 - 2018
stmt1 = """
select gvkey, bkvlps, datadate, cik
from comp.funda
where  datadate in {}
""".format(DATE_list)

gvkey_list = conn.raw_sql(stmt1)
print(gvkey_list)

gvkey_list = gvkey_list[np.isfinite(gvkey_list['bkvlps'])]
print(gvkey_list)

NameError: name 'conn' is not defined

In [ ]:
# select keydata from linktable
stmt2= """
select gvkey, lpermno as permno
from crsp.ccmxpf_linktable
where gvkey in {}
""".format(tuple(gvkey_list['gvkey']))

ccm = conn.raw_sql(stmt2)
print(ccm)

In [ ]:
ccm = ccm[np.isfinite(ccm['permno'])]
ccm['permno']=ccm['permno'].astype('int64', copy=False)
print(ccm)

In [ ]:
ccm2 = pd.DataFrame()

for i in range(len(fd_list)):
    stmt3="""
    select permno, ret, retx, date(date) as date
    FROM crsp.dsf
    where permno = {}
    and  date < date '{}' + integer '7'
    and date > date '{}' - integer '7'
    """.format(ccm['permno'].values[i],fd_list[i],fd_list[i])
    ccm2=ccm2.append( conn.raw_sql(stmt3))
    
print(ccm2)

In [ ]:
# Merge the data
ccm3 = pd.merge(ccm1, ccm2, how="right", on=['permno'])
print(ccm3)